# 🧱 6. 패딩 (Padding): 길이 맞추기

> **"제각각인 문장 길이를 똑같이 맞춰줘!"**

컴퓨터(AI 모델)는 행렬 연산을 하기 때문에, 들어오는 **데이터들의 길이가 모두 똑같아야 해**.

- 문장 1: `[1, 2, 3]` (길이 3)
- 문장 2: `[4, 5, 6, 7, 8]` (길이 5)

이 상태로는 계산 불가! ❌
짧은 문장의 빈 공간을 **0**으로 채워서 길이를 5로 맞춰주는 게 **패딩(Padding)**이야.

- 문장 1: `[0, 0, 1, 2, 3]` (길이 5, 앞쪽 패딩) ✅

---

## 1. Keras로 패딩하기 📦

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 아까 만든 정수 인코딩 결과라고 치자
sequences = [
    [1, 2, 3],          # 짧은 문장
    [4, 5, 6, 7, 8],    # 긴 문장
    [9]                 # 아주 짧은 문장
]

# 1. 기본 패딩 (가장 긴 문장에 맞춰서, 앞쪽을 0으로 채움)
padded = pad_sequences(sequences)
print("기본 패딩:\n", padded)

기본 패딩:
 [[0 0 1 2 3]
 [4 5 6 7 8]
 [0 0 0 0 9]]


## 2. 옵션 조절하기 🎛️

- `padding='post'`: 뒤쪽을 0으로 채우기 (보통 RNN 계열은 'pre'가 좋지만, 상황 따라 씀)
- `maxlen=3`: 길이 제한하기 (너무 긴 건 잘라버려!)

In [3]:
# 뒤쪽 채우기 (Post Padding)
padded_post = pad_sequences(sequences, padding='post')
print("뒤쪽 패딩:\n", padded_post)

# 길이 제한 + 잘라내기 (Truncating)
# maxlen=3으로 하면 길이가 5인 문장은 잘려나감.
padded_limit = pad_sequences(sequences, maxlen=3, truncating='post')
print("길이 제한(3):\n", padded_limit)

뒤쪽 패딩:
 [[1 2 3 0 0]
 [4 5 6 7 8]
 [9 0 0 0 0]]
길이 제한(3):
 [[1 2 3]
 [4 5 6]
 [0 0 9]]


### ⚠️ 중요 포인트
0은 **'아무 의미 없는 빈 공간'**이라는 뜻이야.
AI 모델에게도 나중에 "야, 0번은 무시해!"라고 알려주게 돼. (Masking)

## 🎓 마무리 퀴즈

**Q. 길이가 제각각인 문장들을 행렬 연산이 가능한 직사각형 형태로 만들기 위해 채워넣는 숫자는?**
1. 0
2. 7
3. 99

<details>
<summary>정답 확인</summary>

**1번 (0)**

</details>